In [7]:
!pip install scikit-learn==1.3.2 pandas==2.2.2 numpy==1.26.4 tqdm==4.66.1 joblib==1.3.2 mlflow==2.7.0 dvc==2.33.1 fastapi==0.98.0 uvicorn==0.22.0 gradio==3.40.1 python-multipart==0.0.6

  Using cached scikit_learn-1.3.2-cp312-cp312-win_amd64.whl.metadata (11 kB)
  Using cached pandas-2.2.2-cp312-cp312-win_amd64.whl.metadata (19 kB)
  Using cached numpy-1.26.4-cp312-cp312-win_amd64.whl.metadata (61 kB)
  Using cached tqdm-4.66.1-py3-none-any.whl.metadata (57 kB)
  Using cached joblib-1.3.2-py3-none-any.whl.metadata (5.4 kB)
  Using cached mlflow-2.7.0-py3-none-any.whl.metadata (12 kB)
  Using cached dvc-2.33.1-py3-none-any.whl.metadata (19 kB)
  Using cached fastapi-0.98.0-py3-none-any.whl.metadata (22 kB)
  Using cached uvicorn-0.22.0-py3-none-any.whl.metadata (6.3 kB)
  Using cached gradio-3.40.1-py3-none-any.whl.metadata (17 kB)
  Using cached python_multipart-0.0.6-py3-none-any.whl.metadata (2.5 kB)
     ---------------------------------------- 0.0/1.0 MB ? eta -:--:--
     ---------------------------------------- 1.0/1.0 MB 9.9 MB/s eta 0:00:00
  Installing build dependencies: started
  Installing build dependencies: still running...
  Installing build dependencie

  error: subprocess-exited-with-error
  
  exit code: 1
  
  [354 lines of output]
  <string>:34: UserWarning: pkg_resources is deprecated as an API. See https://setuptools.pypa.io/en/latest/pkg_resources.html. The pkg_resources package is slated for removal as early as 2025-11-30. Refrain from using this package or pin to Setuptools<81.
  C:\Users\gvtit\AppData\Local\Temp\pip-build-env-fwozcoc3\overlay\Lib\site-packages\setuptools\_distutils\dist.py:289: UserWarning: Unknown distribution option: 'tests_require'
    warnings.warn(msg)
  C:\Users\gvtit\AppData\Local\Temp\pip-build-env-fwozcoc3\overlay\Lib\site-packages\setuptools\_distutils\dist.py:289: UserWarning: Unknown distribution option: 'test_suite'
    warnings.warn(msg)
  toml section missing WindowsPath('pyproject.toml') does not contain a tool.setuptools_scm section
  toml section missing WindowsPath('pyproject.toml') does not contain a tool.setuptools_scm section
  C:\Users\gvtit\AppData\Local\Temp\pip-build-env-fwozcoc3\ov

In [9]:
!pip install nltk

   ---------------------------------------- 0.0/1.5 MB ? eta -:--:--
   ---------------------------------------- 1.5/1.5 MB 11.5 MB/s eta 0:00:00



[notice] A new release of pip is available: 25.0.1 -> 25.3
[notice] To update, run: python.exe -m pip install --upgrade pip


In [15]:
import os
import re
import pandas as pd
import nltk
import joblib

from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer

from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, classification_report

# --------------------------------------------------------------------
# 1. Download nltk resources
# --------------------------------------------------------------------
nltk.download("stopwords")
nltk.download("wordnet")

stop_words = set(stopwords.words("english"))
lemmatizer = WordNetLemmatizer()

def clean_text(text):
    text = text.lower()
    text = re.sub(r"[^a-zA-Z\s]", " ", text)
    words = text.split()
    words = [w for w in words if w not in stop_words]
    words = [lemmatizer.lemmatize(w) for w in words]
    return " ".join(words)

# --------------------------------------------------------------------
# 2. Load dataset
# --------------------------------------------------------------------
df = pd.read_csv("data/realistic_invoices.csv")
df["text"] = df["text"].astype(str)
df["cleaned"] = df["text"].apply(clean_text)

X = df["cleaned"]
y = df["label"]

# --------------------------------------------------------------------
# 3. Train-test split
# --------------------------------------------------------------------
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42, stratify=y
)

# --------------------------------------------------------------------
# 4. TF-IDF vectorizer
# --------------------------------------------------------------------
vectorizer = TfidfVectorizer(max_features=5000, ngram_range=(1, 2))
X_train_vec = vectorizer.fit_transform(X_train)
X_test_vec = vectorizer.transform(X_test)

# --------------------------------------------------------------------
# 5. Train Logistic Regression
# --------------------------------------------------------------------
model = LogisticRegression(max_iter=300)
model.fit(X_train_vec, y_train)

# --------------------------------------------------------------------
# 6. Evaluate
# --------------------------------------------------------------------
y_pred = model.predict(X_test_vec)

print("Accuracy:", accuracy_score(y_test, y_pred))
print("\nClassification Report:\n", classification_report(y_test, y_pred))

# --------------------------------------------------------------------
# 7. Save final model + vectorizer
# --------------------------------------------------------------------
os.makedirs("models", exist_ok=True)
joblib.dump(model, "models/invoice_model.pkl")
joblib.dump(vectorizer, "models/invoice_vectorizer.pkl")

print("Saved: invoice_model.pkl + invoice_vectorizer.pkl")

# --------------------------------------------------------------------
# 8. OPTIONAL: Save a checkpoint version (clean)
# --------------------------------------------------------------------
joblib.dump(model, "models/checkpoint_model.pkl")
joblib.dump(vectorizer, "models/checkpoint_vectorizer.pkl")

print("Checkpoint saved successfully!")


[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\gvtit\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\gvtit\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


Accuracy: 0.9102040816326531

Classification Report:
                   precision    recall  f1-score   support

accounts_payable       0.89      0.92      0.90        51
         expense       0.86      0.91      0.89        47
           other       0.89      0.96      0.93        52
  purchase_order       0.98      0.84      0.91        51
             tax       0.95      0.91      0.93        44

        accuracy                           0.91       245
       macro avg       0.91      0.91      0.91       245
    weighted avg       0.91      0.91      0.91       245

Saved: invoice_model.pkl + invoice_vectorizer.pkl
Checkpoint saved successfully!
